# Diabetes Patients Early Readmissions Prediction

**Authors:** [Peter Macinec](https://github.com/pmacinec), [Frantisek Sefcik](https://github.com/FrantisekSefcik)

## Model Selection

In this jupyter notebook, we will try multiple models for prediction and choose the best fitting one.

### Setup and import libraries

In [8]:
# Automatically reloading imported modules
%load_ext autoreload
%autoreload 2

In [9]:
import sys
sys.path.append('..')

# Supress libraries deprecation import warnings
import warnings
warnings.filterwarnings('ignore')

In [10]:
import pandas as pd

from src.preprocessing.helpers import describe_dataset

### Load train and test data

In [6]:
X_train = pd.read_csv('../data/X_train.csv')
X_test = pd.read_csv('../data/X_test.csv')
y_train = pd.read_csv('../data/y_train.csv')
y_test = pd.read_csv('../data/y_test.csv')

In [28]:
describe_dataset(X_train, X_test, y_train, y_test)

Number of train data:  81409
Number of test data:   20353
With  58  features.
In  2  classes:
               0 :  72314  samples,  0.89%
               1 :  9094  samples,  0.11%
